<a href="https://colab.research.google.com/github/hikmatfarhat-ndu/veronica-thesis/blob/master/keras_cifar10-generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolution Network

Convolution Neural Networks (CNN) have been very successful especially when modeling images. In this notebook we introduce CNNs and use Keras to learn the CIFAR10 data set

In [ ]:
%%bash
fileid="1p8-24XAnVxRZCASVhHZobraWT8HaRqyP&export=download" 
filename="cifar10.zip"
curl -L -c cookies.txt 'https://docs.google.com/uc?export=download&id='$fileid | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1/p' > confirm.txt

curl -L -b cookies.txt -o $filename 'https://docs.google.com/uc?export=download&id='$fileid'&confirm='$(<confirm.txt)

rm -f confirm.txt cookies.txt

In [ ]:
!unzip -q cifar10.zip

### Packages

In [ ]:
import tensorflow as tf 
import numpy as np 
import matplotlib.pyplot as plt
from tensorflow.keras import models,layers
from tensorflow.keras.utils import Sequence
#from tensorflow.python.keras.utils import data_utils
import math
import os


In [ ]:
train_dir="/content/cifar10/cifar10/train"
test_dir="/content/cifar10/cifar10/test"

cats=os.listdir(train_dir)
print(cats)
train_figs=np.array([ list(map((train_dir+"/"+cats[i]+"/").__add__,os.listdir(train_dir+"/"+cats[i]))) for i in range(0,10)])
test_figs=np.array([ list(map((test_dir+"/"+cats[i]+"/").__add__,os.listdir(test_dir+"/"+cats[i]))) for i in range(0,10)])

In [ ]:
test_figs=np.transpose(test_figs)
test_figs=np.hstack(test_figs)
train_figs=np.transpose(train_figs)
train_figs=np.hstack(train_figs)
print(train_figs.shape)
train_labels=np.array([ i%10 for i in range(0,len(train_figs))])
test_labels=np.array([ i%10 for i in range(0,len(test_figs))])

In [ ]:
#class CIFAR10Sequence(data_utils.Sequence):
class CIFAR10Sequence(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) *
        self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) *
        self.batch_size]

        return np.array([plt.imread(filename) for filename in batch_x]),batch_y

In [ ]:
batch_size=128
train_dataset=CIFAR10Sequence(train_figs,train_labels,batch_size)
test_dataset=CIFAR10Sequence(test_figs,test_labels,batch_size)
(f,l)=train_dataset.__getitem__(0)
f.shape

In [ ]:
def createModel():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Conv2D(64, (3, 3),  activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())

    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10,activation='softmax'))
    
    return model

In [ ]:
model=createModel()
#tf.keras.utils.plot_model(model,show_shapes=True)
model.summary()

## Optimization

Keras can use many optimization method. In this notebook we use the __Adam__ method which can be described loosely as __adaptive__ gradient descent.

Also since the labels are __NOT__ in one_hot_encoding we use the "Sparse" version of the crossentropy loss: __SparseCategoricalCrossentropy__. Finally, if we don't specify from_logits=False then the loss function would compute softwmax before computing the loss. Since we are computing softwmax in our model already we turn this step off by specifying from_logits=False

In [ ]:
# if we don't use softmax in the last layer, i.e. if the output of the
# model is NOT probabilities then use from_logits=True
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

#history = model.fit(img_train,label_train, batch_size=128,epochs=10, 
#                    validation_data=(img_test, label_test))
history=model.fit(train_dataset,batch_size=batch_size,epochs=10)


### Testing the Accuracy

In [ ]:
_,test_accuracy=model.evaluate(test_dataset)